Créditos: Esse notebook é adaptado dos exemplos fornecidos pelo professor Eduardo Bezerra e códigos utilizados no T1.
- Link para o vídeo: XX

## Objetivo do trabalho:
* Apresentar implementações e descrevê-las com detalhes
* Apresentar os resultados
* Explicar em vídeo

# 1 Engenharia de Features

### Resumo:
* Classificação | 1500 exemplos | 11 atributos | credtrain.txt
* Preprocessamentos: ordinal + count enconding e normalização
* Mining: GradientBoostingClassifier

<s> Codificação Target </s>

Tentei utilizar codificação target, mas não gerou nada. A seguinte informação apareceu: "Warning: No categorical columns found. Calling 'transform' will only return input data."

## Seleção dos dados
Primeiramente é necessário baixar a base de dados

In [2]:
import pandas as pd
nome_colunas = ['ESCT', 'NDEP', 'RENDA', 'TIPOR', 'VBEM', 'NPARC', 'VPARC', 'TEL', 'IDADE', 'RESMS', 'ENTRADA', 'CLASSE']
data = pd.read_csv('cic1205/data/credtrain.txt', sep='\t', names=nome_colunas)

* O argumento **sep='\t'** detecta separações por tab
* O argumento **names=nome_colunas** foi utilizado para nomear as colunas. Para isso foi criada uma array com os nomes das colunas e passada para o parâmetro

## Preprocessamento
Agora tratarei os atributos categóricos e não categóricos.
* Categóricos: Serão codificados em novas variáveis usando ordinal e target encoding
* Não categóricos: Serão normalizados
* Ambos serão unidos num dataframe final

In [3]:
# Lista de variáveis categóricas
variaveis_categoricas = ['ESCT', 'NDEP', 'TIPOR', 'TEL']

# Lista de variáveis numéricas
variaveis_numericas = ['RENDA', 'VBEM', 'NPARC', 'VPARC', 'IDADE', 'RESMS', 'ENTRADA']

# Dataframe com variaveis categoricas
data_cat = data[variaveis_categoricas]

# Dataframe com variaveis numericas
data_num = data[variaveis_numericas]

### Normalização

In [4]:
from sklearn.preprocessing import MinMaxScaler  # Importando biblioteca de normalização Min-Max
scaler = MinMaxScaler()                         # Objeto que realiza a normalização
scaler.fit(data_num)                            # Ajustando o objeto aos dados

def normalizar(input_df):
    data_num_normalized = pd.DataFrame(             # Cria um dataframe que recebe como input um ndarray (numpy)
    scaler.transform(input_df), columns=input_df.columns # O nd-array é retornado pelo método transform
    )
    return data_num_normalized

data_num_normalized = normalizar(data_num)

### Codificação Ordinal

In [5]:
from sklearn.preprocessing import OrdinalEncoder

# Criando o objeto codificador ordinal
ordinal_encoder = OrdinalEncoder()

# Ajustando o objeto aos dados
ordinal_encoder.fit(data_cat)

# Função para codificar os dados usando Ordinal Encoding
def codificar_ordinal(input_df):
    nparray_encoded = ordinal_encoder.transform(input_df)
    data_cat_encoded = pd.DataFrame(nparray_encoded, columns=input_df.columns)
    return data_cat_encoded

# Codificando os dados
data_cat_encoded_ordinal = codificar_ordinal(data_cat)

### Codificação Contagem

Retornará quantas vezes um elemento aparece no dataset (pode ser útil para contar eventos etc)

In [16]:
from category_encoders import CountEncoder

# Criando o objeto codificador de contagem
count_encoder = CountEncoder()

# Ajustando o objeto aos dados
count_encoder.fit(data_cat)

# Codificando os dados
data_cat_encoded_count = count_encoder.transform(data_cat)

### Concatenando os dataframes preprocessados

In [17]:
data_preprocessed_ordinal = pd.concat( [ data_cat_encoded_ordinal, data_num_normalized, data['CLASSE'] ], axis=1 )

In [41]:
data_preprocessed_count = pd.concat( [ data_cat_encoded_count, data_num_normalized, data['CLASSE'] ], axis=1 )

## Mineração

O resultado de GradientBoosting com One-hot encoding **F1 Macro = 0.90**

### Usando Ordinal encoding com GradientBoosting

In [20]:
from sklearn.model_selection import train_test_split # Importa classe que faz amostragem (sampling)

X_train, X_validation, y_train, y_validation = train_test_split(data_preprocessed_ordinal.iloc[:,:-1], # features
                                                                data_preprocessed_ordinal.iloc[:,-1],  # labels
                                                                test_size=0.10,                 # % do test set (10%)
                                                                random_state=1)                # seed

In [21]:
from sklearn.ensemble import GradientBoostingClassifier  # Importando Classe de treinamento de Aumento de Gradiente
model = GradientBoostingClassifier(random_state=1)       # Instanciando objeto classificador
model.fit(X_train, y_train)                              # Ajustando o modelo

GradientBoostingClassifier(random_state=1)

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

overfitted_pred = model.predict(X_train)                # Checando se o modelo ajustou aos dados
accuracy = accuracy_score(y_train, overfitted_pred)
print(f"Overfitted Accuracy: {accuracy * 100:.2f}%")                            # Ajustando o modelo

Overfitted Accuracy: 94.07%


In [25]:
y_pred_validation = model.predict(X_validation) # Resultados no set de validação

# Evaluate the model
accuracy = accuracy_score(y_validation, y_pred_validation)
confusion = confusion_matrix(y_validation, y_pred_validation)
classification_rep = classification_report(y_validation, y_pred_validation)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)

Accuracy: 88.67%
Confusion Matrix:
 [[69  3]
 [14 64]]
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.96      0.89        72
           1       0.96      0.82      0.88        78

    accuracy                           0.89       150
   macro avg       0.89      0.89      0.89       150
weighted avg       0.90      0.89      0.89       150



#### Preprocessamento para teste do modelo
Como o dataset de teste está em outro arquivo (credtest.txt), será necessário repetir os passos de preprocessamento usando os objetos já treinados na fase de treinamento do modelo. Os passos são:
* Seleção de dados
* Separação dos atributos numéricos e categóricos
* Utilização dos objetos de preprocessamento já criados para transformar os novos dados (scaler e codificador)
* Junção do dataset
* Teste final

In [36]:
test_data = pd.read_csv('cic1205/data/credtest.txt', sep='\t', names=nome_colunas) # Seleção de dados
test_data_cat = test_data[variaveis_categoricas] # Separação dos atributos categóricos
test_data_num = test_data[variaveis_numericas]   # Separação dos atributos categóricos
test_data_cat_encoded = codificar_ordinal(test_data_cat)
test_data_num_normalized = normalizar(test_data_num)   # Normalização
test_data_preprocessed = pd.concat( [ test_data_cat_encoded, test_data_num_normalized, test_data['CLASSE'] ], axis=1 ) # Junção
X_test = test_data_preprocessed.iloc[:,:-1] # Conjuntos de dados
y_test = test_data_preprocessed.iloc[:,-1]  # Classificações

In [37]:
y_pred = model.predict(X_test) # Resultados no set de testes

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)

Accuracy: 89.43%
Confusion Matrix:
 [[293  13]
 [ 48 223]]
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.96      0.91       306
           1       0.94      0.82      0.88       271

    accuracy                           0.89       577
   macro avg       0.90      0.89      0.89       577
weighted avg       0.90      0.89      0.89       577



### Usando Count encoding com GradientBoosting

In [42]:
from sklearn.model_selection import train_test_split # Importa classe que faz amostragem (sampling)

X_train, X_validation, y_train, y_validation = train_test_split(data_preprocessed_count.iloc[:,:-1], # features
                                                                data_preprocessed_count.iloc[:,-1],  # labels
                                                                test_size=0.10,                 # % do test set (10%)
                                                                random_state=1)                # seed

In [43]:
from sklearn.ensemble import GradientBoostingClassifier  # Importando Classe de treinamento de Aumento de Gradiente
model = GradientBoostingClassifier(random_state=1)       # Instanciando objeto classificador
model.fit(X_train, y_train)                              # Ajustando o modelo

GradientBoostingClassifier(random_state=1)

In [44]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

overfitted_pred = model.predict(X_train)                # Checando se o modelo ajustou aos dados
accuracy = accuracy_score(y_train, overfitted_pred)
print(f"Overfitted Accuracy: {accuracy * 100:.2f}%")                            # Ajustando o modelo

Overfitted Accuracy: 94.07%


In [45]:
y_pred_validation = model.predict(X_validation) # Resultados no set de validação

# Evaluate the model
accuracy = accuracy_score(y_validation, y_pred_validation)
confusion = confusion_matrix(y_validation, y_pred_validation)
classification_rep = classification_report(y_validation, y_pred_validation)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)

Accuracy: 88.67%
Confusion Matrix:
 [[69  3]
 [14 64]]
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.96      0.89        72
           1       0.96      0.82      0.88        78

    accuracy                           0.89       150
   macro avg       0.89      0.89      0.89       150
weighted avg       0.90      0.89      0.89       150



#### Preprocessamento para teste do modelo
Como o dataset de teste está em outro arquivo (credtest.txt), será necessário repetir os passos de preprocessamento usando os objetos já treinados na fase de treinamento do modelo. Os passos são:
* Seleção de dados
* Separação dos atributos numéricos e categóricos
* Utilização dos objetos de preprocessamento já criados para transformar os novos dados (scaler e codificador)
* Junção do dataset
* Teste final

In [46]:
test_data = pd.read_csv('cic1205/data/credtest.txt', sep='\t', names=nome_colunas) # Seleção de dados
test_data_cat = test_data[variaveis_categoricas] # Separação dos atributos categóricos
test_data_num = test_data[variaveis_numericas]   # Separação dos atributos categóricos
test_data_cat_encoded = count_encoder.transform(test_data_cat)
test_data_num_normalized = normalizar(test_data_num)   # Normalização
test_data_preprocessed = pd.concat( [ test_data_cat_encoded, test_data_num_normalized, test_data['CLASSE'] ], axis=1 ) # Junção
X_test = test_data_preprocessed.iloc[:,:-1] # Conjuntos de dados
y_test = test_data_preprocessed.iloc[:,-1]  # Classificações

In [47]:
y_pred = model.predict(X_test) # Resultados no set de testes

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)

Accuracy: 89.43%
Confusion Matrix:
 [[293  13]
 [ 48 223]]
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.96      0.91       306
           1       0.94      0.82      0.88       271

    accuracy                           0.89       577
   macro avg       0.90      0.89      0.89       577
weighted avg       0.90      0.89      0.89       577



### Análise comparativa de resultados (F1-Macro avg):
- One-hot encoding: 0.90
- Ordinal encoding: 0.89
- Count encoding: 0.89

# 2 Validação Cruzada

### Resumo:
* Classificação | A602.pickle, A621.pickle, A627.pickle, A636.pickle, A652.pickle
* Preprocessamentos: binarização
* Mining: GradientBoostingClassifier com CrossValidation
* Tarefa: Hyperparameter Tunning com Stritified5Fold Validation

In [71]:
import numpy as np
import pickle
fileA602 = open('cic1205/data/atmoseer/A602.pickle' , 'rb')
fileA621 = open('cic1205/data/atmoseer/A621.pickle' , 'rb')
fileA627 = open('cic1205/data/atmoseer/A627.pickle' , 'rb')
fileA636 = open('cic1205/data/atmoseer/A636.pickle' , 'rb')
fileA652 = open('cic1205/data/atmoseer/A652.pickle' , 'rb')

### Dataset A602

In [72]:
(X_train , y_train , X_val , y_val , X_test , y_test) = pickle.load(fileA602)

In [81]:
#Unindo dataset de treino e validação
X_train = np.concatenate([X_train, X_val], axis=0)
y_train = np.concatenate([y_train, y_val])

# Preprocessamento (Botando 1 tudo que é diferente de 0 como informado no grupo)
y_train[y_train != 0] = 1
y_test[y_test != 0] = 1
# Transformando 2D array em 1D array
y_train = y_train.flatten()
y_test = y_test.flatten()

In [85]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # 5 folds

# Definindo parametros
param_dist = {
    'n_estimators': [25, 50, 100],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [2, 3, 4],
    'min_samples_split': [2],
    'min_samples_leaf': [3, 4, 6],
    'subsample': [0.3, 0.6, 1.0]
}

# Busca randômica
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=10,
    scoring='accuracy',
    cv=cv,
    random_state=42
)

# Fit the randomized search to the data
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Get the best model
best_model = random_search.best_estimator_


Best Hyperparameters: {'subsample': 1.0, 'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 3, 'learning_rate': 0.1}


**Busca 1**:
Best Hyperparameters: {'subsample': 1.0, 'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 3, 'learning_rate': 0.1}

**Busca 2**:
Best Hyperparameters: {'subsample': 1.0, 'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 3, 'learning_rate': 0.1}

In [86]:
# Predição
y_pred = best_model.predict(X_test)

# Classification report conforme requisitado
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)

Accuracy: 88.89%
Confusion Matrix:
 [[633  40]
 [ 45  47]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.93      0.94      0.94       673
         1.0       0.54      0.51      0.53        92

    accuracy                           0.89       765
   macro avg       0.74      0.73      0.73       765
weighted avg       0.89      0.89      0.89       765



#### Resultados A602
- F1 Classe 1 antigo: 0.54
- F1 Classe 1 novo: 0.53

### Dataset A621

In [88]:
(X_train , y_train , X_val , y_val , X_test , y_test) = pickle.load(fileA621)

EOFError: Ran out of input

In [89]:
#Unindo dataset de treino e validação
X_train = np.concatenate([X_train, X_val], axis=0)
y_train = np.concatenate([y_train, y_val])

# Preprocessamento (Botando 1 tudo que é diferente de 0 como informado no grupo)
y_train[y_train != 0] = 1
y_test[y_test != 0] = 1
# Transformando 2D array em 1D array
y_train = y_train.flatten()
y_test = y_test.flatten()

In [90]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # 5 folds

# Definindo parametros
param_dist = {
    'n_estimators': [25, 50, 100],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [2, 3, 4],
    'min_samples_split': [2],
    'min_samples_leaf': [3, 4, 6],
    'subsample': [0.3, 0.6, 1.0]
}

# Busca randômica
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=10,
    scoring='accuracy',
    cv=cv,
    random_state=42
)

# Hora do treinamento
random_search.fit(X_train, y_train)

# Informar melhores parâmetros
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Setar um modelo com os melhores parâmetros
best_model = random_search.best_estimator_


Best Hyperparameters: {'subsample': 1.0, 'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 3, 'learning_rate': 0.1}


**Busca 1**:
Best Hyperparameters: {'subsample': 1.0, 'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 3, 'learning_rate': 0.1}

**Busca 2**:
Best Hyperparameters: {'subsample': 1.0, 'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 3, 'learning_rate': 0.1}

**Busca 3**: Mesmo que busca 2

In [91]:
# Predição
y_pred = best_model.predict(X_test)

# Classification report conforme requisitado
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)

Accuracy: 93.25%
Confusion Matrix:
 [[8945  202]
 [ 478  455]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      0.98      0.96      9147
         1.0       0.69      0.49      0.57       933

    accuracy                           0.93     10080
   macro avg       0.82      0.73      0.77     10080
weighted avg       0.93      0.93      0.93     10080



#### Resultados A621
- F1 Classe 1 antigo: 0.58
- F1 Classe 1 novo: 0.57

### Dataset A627

In [94]:
(X_train , y_train , X_val , y_val , X_test , y_test) = pickle.load(fileA627)

In [95]:
#Unindo dataset de treino e validação
X_train = np.concatenate([X_train, X_val], axis=0)
y_train = np.concatenate([y_train, y_val])

# Preprocessamento (Botando 1 tudo que é diferente de 0 como informado no grupo)
y_train[y_train != 0] = 1
y_test[y_test != 0] = 1
# Transformando 2D array em 1D array
y_train = y_train.flatten()
y_test = y_test.flatten()

In [96]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # 5 folds

# Definindo parametros
param_dist = {
    'n_estimators': [25, 50, 100],
    'learning_rate': [0.1, 1],
    'max_depth': [2, 3, 4],
    'min_samples_split': [2],
    'min_samples_leaf': [3, 4],
    'subsample': [0.6, 1.0]
}

# Busca randômica
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=10,
    scoring='accuracy',
    cv=cv,
    random_state=42
)

# Hora do treinamento
random_search.fit(X_train, y_train)

# Informar melhores parâmetros
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Setar um modelo com os melhores parâmetros
best_model = random_search.best_estimator_


Best Hyperparameters: {'subsample': 0.6, 'n_estimators': 25, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 3, 'learning_rate': 0.1}


**Best Hyperparameters**: {'subsample': 0.6, 'n_estimators': 25, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 3, 'learning_rate': 0.1}

In [97]:
# Predição
y_pred = best_model.predict(X_test)

# Classification report conforme requisitado
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)

Accuracy: 93.25%
Confusion Matrix:
 [[8926  274]
 [ 415  587]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.96      0.97      0.96      9200
         1.0       0.68      0.59      0.63      1002

    accuracy                           0.93     10202
   macro avg       0.82      0.78      0.80     10202
weighted avg       0.93      0.93      0.93     10202



#### Resultados A627
- F1 Classe 1 antigo: 0.62
- F1 Classe 1 novo: 0.63

### Dataset A636

In [98]:
(X_train , y_train , X_val , y_val , X_test , y_test) = pickle.load(fileA636)

In [99]:
#Unindo dataset de treino e validação
X_train = np.concatenate([X_train, X_val], axis=0)
y_train = np.concatenate([y_train, y_val])

# Preprocessamento (Botando 1 tudo que é diferente de 0 como informado no grupo)
y_train[y_train != 0] = 1
y_test[y_test != 0] = 1
# Transformando 2D array em 1D array
y_train = y_train.flatten()
y_test = y_test.flatten()

In [100]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # 5 folds

# Definindo parametros
param_dist = {
    'n_estimators': [25, 50, 100],
    'learning_rate': [0.1, 1],
    'max_depth': [2, 3, 4],
    'min_samples_split': [2],
    'min_samples_leaf': [3, 4],
    'subsample': [0.6, 1.0]
}

# Busca randômica
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=10,
    scoring='accuracy',
    cv=cv,
    random_state=42
)

# Hora do treinamento
random_search.fit(X_train, y_train)

# Informar melhores parâmetros
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Setar um modelo com os melhores parâmetros
best_model = random_search.best_estimator_


Best Hyperparameters: {'subsample': 0.6, 'n_estimators': 25, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 2, 'learning_rate': 0.1}


**Best Hyperparameters**: {'subsample': 0.6, 'n_estimators': 25, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 2, 'learning_rate': 0.1}

In [101]:
# Predição
y_pred = best_model.predict(X_test)

# Classification report conforme requisitado
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)

Accuracy: 92.46%
Confusion Matrix:
 [[8850  167]
 [ 597  515]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.98      0.96      9017
         1.0       0.76      0.46      0.57      1112

    accuracy                           0.92     10129
   macro avg       0.85      0.72      0.77     10129
weighted avg       0.92      0.92      0.92     10129



#### Resultados A636
- F1 Classe 1 antigo: 0.62
- F1 Classe 1 novo: 0.57

### Dataset A652

In [102]:
(X_train , y_train , X_val , y_val , X_test , y_test) = pickle.load(fileA652)

In [103]:
#Unindo dataset de treino e validação
X_train = np.concatenate([X_train, X_val], axis=0)
y_train = np.concatenate([y_train, y_val])

# Preprocessamento (Botando 1 tudo que é diferente de 0 como informado no grupo)
y_train[y_train != 0] = 1
y_test[y_test != 0] = 1
# Transformando 2D array em 1D array
y_train = y_train.flatten()
y_test = y_test.flatten()

In [104]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # 5 folds

# Definindo parametros
param_dist = {
    'n_estimators': [25, 50, 100],
    'learning_rate': [0.1, 1],
    'max_depth': [2, 3, 4],
    'min_samples_split': [2],
    'min_samples_leaf': [3, 4],
    'subsample': [0.6, 1.0]
}

# Busca randômica
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=10,
    scoring='accuracy',
    cv=cv,
    random_state=42
)

# Hora do treinamento
random_search.fit(X_train, y_train)

# Informar melhores parâmetros
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Setar um modelo com os melhores parâmetros
best_model = random_search.best_estimator_


Best Hyperparameters: {'subsample': 0.6, 'n_estimators': 25, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 2, 'learning_rate': 0.1}


**Best Hyperparameters**: {'subsample': 0.6, 'n_estimators': 25, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 2, 'learning_rate': 0.1}

In [101]:
# Predição
y_pred = best_model.predict(X_test)

# Classification report conforme requisitado
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)

Accuracy: 92.46%
Confusion Matrix:
 [[8850  167]
 [ 597  515]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.98      0.96      9017
         1.0       0.76      0.46      0.57      1112

    accuracy                           0.92     10129
   macro avg       0.85      0.72      0.77     10129
weighted avg       0.92      0.92      0.92     10129



#### Resultados A652
- F1 Classe 1 antigo: 0.50
- F1 Classe 1 novo: 0.57